In [99]:
from bs4 import BeautifulSoup
import nltk
from nltk.stem import WordNetLemmatizer
# nltk.download('punkt')
# nltk.download('wordnet')
import re
import numpy as np

In [53]:
positive = BeautifulSoup(open('electronics/positive.review', encoding='utf8')).find_all('review_text')
positive_text = [review.text.strip() for review in positive]

negative = BeautifulSoup(open('electronics/negative.review', encoding='utf8')).find_all('review_text')
negative_text = [review.text.strip() for review in negative]

In [33]:
stopwords = [word.rstrip() for word in open('stopwords.txt')]
stopwords[:10]

['a',
 'about',
 'above',
 'across',
 'after',
 'again',
 'against',
 'all',
 'almost',
 'alone']

In [93]:
tokenizer(positive_text[1])

['back-ups',
 'recommendation',
 'employee',
 'mine',
 'month',
 'functioned',
 'properly',
 'unexpected',
 'power',
 'interruption',
 'gladly',
 'arises',
 'plug',
 'spacing',
 'power',
 'adapter',
 'simple',
 'design',
 'cord',
 'line',
 'conditioning',
 'usually',
 'expensive',
 'option']

In [84]:
lemmatizer = WordNetLemmatizer()

def tokenizer(review):
    tokens = nltk.tokenize.word_tokenize(review.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # 過濾文法變化
    tokens = [token for token in tokens if len(token) > 3 and token not in stopwords] # 過濾掉過短單字 & stopwords
    return tokens

In [87]:
word_idx_map = {}
current_idx = 0
pos_tokenized = []
neg_tokenized = []
origin_reviews = []

for review in positive_text:
    origin_reviews.append(review)
    tokens = tokenizer(review)
    pos_tokenized.append(tokens)
    for token in tokens:
        if token not in word_idx_map:
            word_idx_map[token] = current_idx
            current_idx += 1

for review in negative_text:
    origin_reviews.append(review)
    tokens = tokenizer(review)
    pos_tokenized.append(tokens)
    for token in tokens:
        if token not in word_idx_map:
            word_idx_map[token] = current_idx
            current_idx += 1

print("len(word_idx_map):", len(word_idx_map))

len(word_idx_map): 10147


In [94]:
word_idx_map['amazon'], word_idx_map['recommendation']

(30, 33)

In [95]:
def tokens_to_vector(tokens, label):
    matrix = np.zeros(len(word_idx_map) + 1) # 最後一個是標記
    for token in tokens:
        idx = word_idx_map[token]
        matrix[idx] += 1
    matrix = matrix / matrix.sum() # 正規化數據提升未來準確度
    matrix[-1] = label
    return matrix

In [101]:
N = len(pos_tokenized) + len(neg_tokenized)   # (N x D+1) 矩陣 (擺在一塊將來便於 shuffle)

data = np.zeros((N, len(word_idx_map) + 1))

idx = 0

for tokens in pos_tokenized:
    data[idx] = tokens_to_vector(tokens, 1)
    idx += 1

for tokens in neg_tokenized:
    data[idx] = tokens_to_vector(tokens, 0)
    idx += 1